# **Pre Classification Analysis** 

In [1]:
#import sys
#sys.path
#sys.path.append('c:\\Users\\3leso\\Documents\\Elena\\Uni\\MasterThesis\\MasterThesis_final\\code\\faultana')

### Set file paths

In [1]:
subreddit_here = "politics"
subreddit = "politics"


output_path = f"output/{subreddit_here}_new_preprocessing_com_rep.csv"



In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import matplotlib.pyplot as plt
import seaborn as sns
#import community as community_louvain

import zstandard
import io

from sklearn.model_selection import train_test_split
from scipy import sparse
import random
from itertools import combinations
import collections
import multiprocessing
import copy
import time
#import gurobipy
#from gurobipy import *

pd.options.mode.chained_assignment = None

In [4]:
from preprocessing_functions import import_subreddit
from preprocessing_functions import filter_subreddit
from preprocessing_functions import aggregate_interactions_directed
from preprocessing_functions import get_counts_and_shares
from preprocessing_functions import find_max
from preprocessing_functions import build_undirected
from preprocessing_functions import get_submissions
from preprocessing_functions import get_comments_and_replies
from preprocessing_functions import assign_types
from preprocessing_functions import find_relevant_submissions
from preprocessing_functions import match_reply_comment_submission
from preprocessing_functions import preprocess_texts
from preprocessing_functions import import_merge_predicted_labels
from preprocessing_functions import import_subreddit_to_parquet

***

## **Data Import**

In [ ]:
# write comments 1 to parquet
#import_subreddit_to_parquet(file_comments, "../../../data/reddit/politics_comments_parquet_new.zst", chunk_size=100_000, data_type="comments")

# write submissions 1 to parquet
#import_subreddit_to_parquet(file_submissions, "../../../data/reddit/politics_submissions_parquet_new.zst", chunk_size=100_000, data_type="submissions")



Written 100000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 200000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 300000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 400000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 500000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 600000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 700000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 800000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 900000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 1000000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 1100000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 1200000 rows to ../../../data/reddit/politics_comments_parquet_new.zst
Written 1300000 rows to ../../../data/reddit/politics_comment

In [5]:
# import
data_comments = pd.read_parquet("politics_comments_parquet_new.zst", engine="pyarrow")
data_submissions = pd.read_parquet("politics_submissions_parquet_new.zst", engine="pyarrow")

In [6]:
# make df
df_comments = pd.DataFrame(data_comments)
df_submissions = pd.DataFrame(data_submissions)

In [7]:
# submissions
# filter relevant columns

submissions = get_submissions(df_submissions)
submissions

,author,created_utc,id,num_comments,score,send_replies,stickied,subreddit,subreddit_id,title,name,likes,ups,category,selftext,type,date
0,db2,1.186378e+09,2cnih,0,0,True,False,politics,t5_2cneq,Republican hopefuls wade into terrorist strike...,None,None,None,None,,submission,2007-08-06
1,goldfish1man,1.186378e+09,2cnj0,0,0,True,False,politics,t5_2cneq,Poem From Guantanamo,None,None,None,None,,submission,2007-08-06
2,Xiphorian,1.186379e+09,2cnnc,10,66,True,False,politics,t5_2cneq,"Smashing interview by Ron Paul on FOX! ""Just ...",None,None,None,None,,submission,2007-08-06
3,bakosdelowang,1.186380e+09,2cnqd,0,5,True,False,politics,t5_2cneq,House approves $460B Pentagon budget... better...,None,None,None,None,,submission,2007-08-06
4,rayhoffman,1.186381e+09,2cnsc,0,0,True,False,politics,t5_2cneq,Ron Paul Smear Job,None,None,None,None,,submission,2007-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4609223,invertedspheres,1.704061e+09,18vihft,0,1,True,False,politics,t5_2cneq,Judge orders release of over 150 names of peop...,t3_18vihft,None,1,None,[removed],submission,2023-12-31
4609224,TheTruthTalker800,1.704062e+09,18vizm7,691,3035,True,False,politics,t5_2cneq,Plane from Texas drops off more than 300 migra...,t3_18vizm7,None,3035,None,,submission,2023-12-31
4609225,iguess12,1.704065e+09,18vjps0,71,1,True,False,politics,t5_2cneq,Migrant crossings at U.S. southern border reac...,t3_18vjps0,None,1,None,,submission,2023-12-31
4609226,[deleted],1.704065e+09,18vju0t,0,1,True,False,politics,t5_2cneq,Google Experiments With “Faster and More Adapt...,t3_18vju0t,None,1,None,[removed],submission,2023-12-31


In [8]:
# comments
# select relevant columns only

df_comments_replies = get_comments_and_replies(df_comments)
df_comments_replies

,author,created_utc,parent_id,subreddit_id,body,link_id,name,id,ups,score,downs,subreddit,likes,replies,stickied,send_replies,comment_type,date
0,lojomofo,1.186380e+09,2cnnc,2cneq,"DAMN, she's a fox! Also I love his retort to t...",2cnnc,None,c2cnq9,6,6,None,politics,None,None,False,None,None,2007-08-06
1,[deleted],1.186380e+09,2cnnc,2cneq,[removed],2cnnc,None,c2cnqr,0,0,None,politics,None,None,False,None,None,2007-08-06
2,shadowsurge,1.186382e+09,2cnti,2cneq,"Personally I'd prefer both, its just that I ha...",2cnti,None,c2cnwe,12,12,None,politics,None,None,False,None,None,2007-08-06
3,wats8976,1.186383e+09,2cnvc,2cneq,What no?,2cnvc,None,c2cnxw,-1,-1,None,politics,None,None,False,None,None,2007-08-06
4,cl3ft,1.186383e+09,2cnvc,2cneq,"well thought out argument there, nigh on irref...",2cnvc,None,c2cnyl,4,4,None,politics,None,None,False,None,None,2007-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187786789,sunflowerastronaut,1.704067e+09,kfrnwmw,2cneq,Experts believe the bill will reduce emissions...,18vd4is,kfrou7r,kfrou7r,2,2,0,politics,None,,False,True,None,2023-12-31
187786790,Tario70,1.704067e+09,kfrhll8,2cneq,Hence why I said it’s a shit situation all aro...,18usgjt,kfrouhj,kfrouhj,1,1,0,politics,None,,False,True,None,2023-12-31
187786791,enigmaroboto,1.704067e+09,kfown66,2cneq,Exactly,18usgjt,kfrouno,kfrouno,-2,-2,0,politics,None,,False,True,None,2023-12-31
187786792,ScaryBuilder9886,1.704067e+09,kfrel1k,2cneq,I don't think that's a change in the definitio...,18vgahm,kfrouuj,kfrouuj,1,1,0,politics,None,,False,True,None,2023-12-31


In [9]:
# assign types and extract them to separate dfs
comments, replies = assign_types(df_comments_replies, submissions)

In [10]:
comments # 34591
print("Len comments: ", len(comments))

replies # 16137
print("Len replies: ", len(replies))

Len comments:  51187999
Len replies:  37509265


### Filter relevant submissions

* relevant = above subreddit-average number of comments
* relevant_comments: those with a parent, that is in relevant_submissions

In [11]:
# select relevant submissions (by avg. number of comments)

relevant_submissions, relevant_comments = find_relevant_submissions(df_comments = comments, df_submissions = submissions, submission_id_col = "parent_id", comment_id_col = "id")


print(f"Number of relevant submissions: {len(relevant_submissions)}")
print("Lost submissions: ", len(submissions) - len(relevant_submissions))
print("Lost comments: ", len(comments) - len(relevant_comments))

relevant_submissions

relevant_comments.reset_index(drop=True, inplace=True)
relevant_comments

14.523119730056315
Number of relevant submissions: 557957
Lost submissions:  4051271
Lost comments:  11546171


,author,created_utc,parent_id,subreddit_id,body,link_id,name,id,ups,score,downs,subreddit,likes,replies,stickied,send_replies,comment_type,date,type
0,DudeAsInCool,1.186418e+09,2crk4,2cneq,Space technology has led to a lot of innovatio...,2crk4,None,c2crnk,9,9,None,politics,None,None,False,None,None,2007-08-06,comment
1,mattbert,1.186419e+09,2crk4,2cneq,"Neither ""private operators"" nor the military a...",2crk4,None,c2crti,29,29,None,politics,None,None,False,None,None,2007-08-06,comment
2,yellow_toe_nails,1.186420e+09,2crk4,2cneq,"So what's Ron Paul going to do? Wipe out NASA,...",2crk4,None,c2crzx,-7,-7,None,politics,None,None,False,None,None,2007-08-06,comment
3,rcunningham,1.186422e+09,2crk4,2cneq,"First and foremost, this article is severaly o...",2crk4,None,c2cse1,6,6,None,politics,None,None,False,None,None,2007-08-06,comment
4,[deleted],1.186426e+09,2crk4,2cneq,[removed],2crk4,None,c2ct8q,-3,-3,None,politics,None,None,False,None,None,2007-08-06,comment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39641823,Morgen019,1.704067e+09,18vaa8o,2cneq,So just for shiggles. If Trump isn’t accounta...,18vaa8o,kfrosib,kfrosib,3,3,0,politics,None,,False,True,None,2023-12-31,comment
39641824,StatusKoi,1.704067e+09,18vi8e1,2cneq,Oh how sweet of them.,18vi8e1,kfrosj0,kfrosj0,1,1,0,politics,None,,False,True,None,2023-12-31,comment
39641825,cmcrisp,1.704067e+09,18uuami,2cneq,"Good. That's it, that's all that can be said a...",18uuami,kfrotlu,kfrotlu,2,2,0,politics,None,,False,True,None,2023-12-31,comment
39641826,NoWallaby1548,1.704067e+09,18ttt89,2cneq,And her polling number jumped by 3% in the pri...,18ttt89,kfrou5f,kfrou5f,1,1,0,politics,None,,False,True,None,2023-12-31,comment


### Build df
* Match first level replies to relevant comments to relevant submissions
* Join = 'inner', because only keep replies, that have a parent in the relevant_comments
    * Missings due to replies, that did not find a comment to belong to.
    * This happend, because comments were dropped, if they were a comment to a submission, that had too little comments (below average)
    * Between comments and submissions, there won't be missings, as relevant_submissions were used to filter relevant_comments 

In [12]:
df_left = match_reply_comment_submission(replies, relevant_comments, relevant_submissions, 'left')
df_inner = match_reply_comment_submission(replies, relevant_comments, relevant_submissions, 'inner') # correct join

print("Left Join:", len(df_left))
print("Inner Join: ", len(df_inner))

print("Length of children in left, that are in inner as well: ", len(df_left[df_left['msg_id_child'].isin(df_inner['msg_id_child'])]))
print("Difference: ", len(df_left) - len(df_inner))

Left Join: 35868765
Inner Join:  31541772
Length of children in left, that are in inner as well:  31540724
Difference:  4326993


In [13]:
df_matched = match_reply_comment_submission(replies, relevant_comments, relevant_submissions, 'inner')
df_matched

,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c2crnk,c2crq9,I tend to agree. While I certainly think the ...,politics,1.186419e+09,MediaMoguls,1,1,None,2crk4,...,DudeAsInCool,9,9,None,1.186418e+09,,MediaMoguls,None,24,1.186418e+09
1,c2crnk,c2cso2,&gt;Space technology has led to a lot of innov...,politics,1.186423e+09,bluGill,11,11,None,2crk4,...,DudeAsInCool,9,9,None,1.186418e+09,,MediaMoguls,None,24,1.186418e+09
2,c2crzx,c2cs1i,"You're a dick, but at least you (sort of) unde...",politics,1.186420e+09,MediaMoguls,-1,-1,None,2crk4,...,yellow_toe_nails,-7,-7,None,1.186420e+09,,MediaMoguls,None,24,1.186418e+09
3,c2crti,c2csdf,"NASA wasn't created to further basic science, ...",politics,1.186422e+09,lastexit,16,16,None,2crk4,...,mattbert,29,29,None,1.186419e+09,,MediaMoguls,None,24,1.186418e+09
4,c2crti,c2ctwh,I would recommend people listen to this podcas...,politics,1.186429e+09,jacoplane,5,5,None,2crk4,...,mattbert,29,29,None,1.186419e+09,,MediaMoguls,None,24,1.186418e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31541767,kfrhv0b,kfrksgo,The Department of Justice does not consider th...,politics,1.704065e+09,Ok-Conversation2707,36,36,0,18vizm7,...,theslats,15,15,0,1.704064e+09,,TheTruthTalker800,3035,3035,1.704062e+09
31541768,kfrjbu1,kfrn1f1,Texas literally puts all their newly released ...,politics,1.704066e+09,Helping-ways,5,5,0,18vizm7,...,StopLookListenNow,3,3,0,1.704065e+09,,TheTruthTalker800,3035,3035,1.704062e+09
31541769,kfrh2km,kfrncg6,And arresting those who drive the buses/planes...,politics,1.704067e+09,milehigh73a,105,105,0,18vizm7,...,Fun-Persimmon1207,189,189,0,1.704064e+09,,TheTruthTalker800,3035,3035,1.704062e+09
31541770,kfrh2km,kfrojav,"Why don’t they do this? For real, is there a l...",politics,1.704067e+09,aitamailmaner,-1,-1,0,18vizm7,...,Fun-Persimmon1207,189,189,0,1.704064e+09,,TheTruthTalker800,3035,3035,1.704062e+09


In [14]:
duplicated = df_matched[['body_child', "body_parent", 'subreddit', "msg_id_child", "msg_id_parent", "author_parent", "author_child"]]
duplicated = duplicated[duplicated.duplicated(['body_child', "body_parent", "author_parent", "author_child"], keep=False) == True]
duplicated

,body_child,body_parent,subreddit,msg_id_child,msg_id_parent,author_parent,author_child


### Filter for Subreddit

In [15]:
# filter for specific subreddit --> technically unnecessary
# also converts date-time and label column

df = filter_subreddit(df_matched, subreddit)
df

,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c2crnk,c2crq9,I tend to agree. While I certainly think the ...,politics,2007-08-06 16:47:47,MediaMoguls,1,1,None,2crk4,...,DudeAsInCool,9,9,None,2007-08-06 16:40:14,,MediaMoguls,None,24,2007-08-06 16:30:43
1,c2crnk,c2cso2,&gt;Space technology has led to a lot of innov...,politics,2007-08-06 17:56:21,bluGill,11,11,None,2crk4,...,DudeAsInCool,9,9,None,2007-08-06 16:40:14,,MediaMoguls,None,24,2007-08-06 16:30:43
2,c2crzx,c2cs1i,"You're a dick, but at least you (sort of) unde...",politics,2007-08-06 17:13:53,MediaMoguls,-1,-1,None,2crk4,...,yellow_toe_nails,-7,-7,None,2007-08-06 17:09:49,,MediaMoguls,None,24,2007-08-06 16:30:43
3,c2crti,c2csdf,"NASA wasn't created to further basic science, ...",politics,2007-08-06 17:35:16,lastexit,16,16,None,2crk4,...,mattbert,29,29,None,2007-08-06 16:53:53,,MediaMoguls,None,24,2007-08-06 16:30:43
4,c2crti,c2ctwh,I would recommend people listen to this podcas...,politics,2007-08-06 19:35:17,jacoplane,5,5,None,2crk4,...,mattbert,29,29,None,2007-08-06 16:53:53,,MediaMoguls,None,24,2007-08-06 16:30:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31541767,kfrhv0b,kfrksgo,The Department of Justice does not consider th...,politics,2023-12-31 23:31:18,Ok-Conversation2707,36,36,0,18vizm7,...,theslats,15,15,0,2023-12-31 23:10:26,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
31541768,kfrjbu1,kfrn1f1,Texas literally puts all their newly released ...,politics,2023-12-31 23:46:51,Helping-ways,5,5,0,18vizm7,...,StopLookListenNow,3,3,0,2023-12-31 23:20:55,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
31541769,kfrh2km,kfrncg6,And arresting those who drive the buses/planes...,politics,2023-12-31 23:49:00,milehigh73a,105,105,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
31541770,kfrh2km,kfrojav,"Why don’t they do this? For real, is there a l...",politics,2023-12-31 23:57:21,aitamailmaner,-1,-1,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11



### Select High Quality Texts


* Filtering
    * remove deleted or removed parent and child texts (submissions no problem)
    * remove deleted authors of parents and childs (submissions no problem)
    * remove bodies (parents and childs) whose exact text occurred too often within the subreddit (i.e. bots)
    

* Preprocessing of texts
    * remove user mentions
    * remove subreddit mentions
    * remove URLs
    * remove line breaks
    * remove multiple spaces
    * replace &amp with and
    * (remove brackets and exclamation marks --> better not, they contain context)


* Filtering, second step
    * remove too short replies & comments (submissions no problem)


In [16]:
df['body_child'].value_counts().head(5)

#df[df['body_child'] == "Sorry, your post has been removed because your account does not currently meet the minimum required karma to post in this subreddit.\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AmericanPolitics) if you have any questions or concerns.*"]

body_child
[removed]    1236923
[deleted]    1201965
Yes            18415
Yes.           11935
No             11299
Name: count, dtype: int64

In [17]:
# remove interactions where either comment or reply (or both) are shorter than 5 words

df = preprocess_texts(data = df, length = 5)
df

Length 5 removes 3658304 interactions


,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c2crnk,c2crq9,I tend to agree. While I certainly think the p...,politics,2007-08-06 16:47:47,MediaMoguls,1,1,None,2crk4,...,DudeAsInCool,9,9,None,2007-08-06 16:40:14,,MediaMoguls,None,24,2007-08-06 16:30:43
1,c2crnk,c2cso2,Space technology has led to a lot of innovatio...,politics,2007-08-06 17:56:21,bluGill,11,11,None,2crk4,...,DudeAsInCool,9,9,None,2007-08-06 16:40:14,,MediaMoguls,None,24,2007-08-06 16:30:43
2,c2crzx,c2cs1i,"You're a dick, but at least you sort of unders...",politics,2007-08-06 17:13:53,MediaMoguls,-1,-1,None,2crk4,...,yellow_toe_nails,-7,-7,None,2007-08-06 17:09:49,,MediaMoguls,None,24,2007-08-06 16:30:43
3,c2crti,c2csdf,"NASA wasn't created to further basic science, ...",politics,2007-08-06 17:35:16,lastexit,16,16,None,2crk4,...,mattbert,29,29,None,2007-08-06 16:53:53,,MediaMoguls,None,24,2007-08-06 16:30:43
4,c2crti,c2ctwh,I would recommend people listen to this podcas...,politics,2007-08-06 19:35:17,jacoplane,5,5,None,2crk4,...,mattbert,29,29,None,2007-08-06 16:53:53,,MediaMoguls,None,24,2007-08-06 16:30:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22178089,kfrhv0b,kfrksgo,The Department of Justice does not consider th...,politics,2023-12-31 23:31:18,Ok-Conversation2707,36,36,0,18vizm7,...,theslats,15,15,0,2023-12-31 23:10:26,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178090,kfrjbu1,kfrn1f1,Texas literally puts all their newly released ...,politics,2023-12-31 23:46:51,Helping-ways,5,5,0,18vizm7,...,StopLookListenNow,3,3,0,2023-12-31 23:20:55,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178091,kfrh2km,kfrncg6,And arresting those who drive the busesplanes....,politics,2023-12-31 23:49:00,milehigh73a,105,105,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178092,kfrh2km,kfrojav,"Why dont they do this? For real, is there a la...",politics,2023-12-31 23:57:21,aitamailmaner,-1,-1,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11


In [18]:
#df

duplicated = df[['body_child', "body_parent", 'subreddit', "msg_id_child", "msg_id_parent", "author_parent", "author_child"]]
duplicated = duplicated[duplicated.duplicated(['body_child', "body_parent", "author_parent", "author_child"], keep=False) == True]
duplicated


,body_child,body_parent,subreddit,msg_id_child,msg_id_parent,author_parent,author_child


### Save and export

In [19]:
df

,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c2crnk,c2crq9,I tend to agree. While I certainly think the p...,politics,2007-08-06 16:47:47,MediaMoguls,1,1,None,2crk4,...,DudeAsInCool,9,9,None,2007-08-06 16:40:14,,MediaMoguls,None,24,2007-08-06 16:30:43
1,c2crnk,c2cso2,Space technology has led to a lot of innovatio...,politics,2007-08-06 17:56:21,bluGill,11,11,None,2crk4,...,DudeAsInCool,9,9,None,2007-08-06 16:40:14,,MediaMoguls,None,24,2007-08-06 16:30:43
2,c2crzx,c2cs1i,"You're a dick, but at least you sort of unders...",politics,2007-08-06 17:13:53,MediaMoguls,-1,-1,None,2crk4,...,yellow_toe_nails,-7,-7,None,2007-08-06 17:09:49,,MediaMoguls,None,24,2007-08-06 16:30:43
3,c2crti,c2csdf,"NASA wasn't created to further basic science, ...",politics,2007-08-06 17:35:16,lastexit,16,16,None,2crk4,...,mattbert,29,29,None,2007-08-06 16:53:53,,MediaMoguls,None,24,2007-08-06 16:30:43
4,c2crti,c2ctwh,I would recommend people listen to this podcas...,politics,2007-08-06 19:35:17,jacoplane,5,5,None,2crk4,...,mattbert,29,29,None,2007-08-06 16:53:53,,MediaMoguls,None,24,2007-08-06 16:30:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22178089,kfrhv0b,kfrksgo,The Department of Justice does not consider th...,politics,2023-12-31 23:31:18,Ok-Conversation2707,36,36,0,18vizm7,...,theslats,15,15,0,2023-12-31 23:10:26,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178090,kfrjbu1,kfrn1f1,Texas literally puts all their newly released ...,politics,2023-12-31 23:46:51,Helping-ways,5,5,0,18vizm7,...,StopLookListenNow,3,3,0,2023-12-31 23:20:55,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178091,kfrh2km,kfrncg6,And arresting those who drive the busesplanes....,politics,2023-12-31 23:49:00,milehigh73a,105,105,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178092,kfrh2km,kfrojav,"Why dont they do this? For real, is there a la...",politics,2023-12-31 23:57:21,aitamailmaner,-1,-1,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11


In [20]:
df.to_csv(output_path, index = False)
df = pd.read_csv(output_path)
df

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [30]:
# to zip

import zipfile
zip_filename = f"output/{subreddit_here}_new_preprocessing_com_rep.zip"

#with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, allowZip64=True) as zf:
#    with zf.open(output_path, 'w') as f:
#        df.to_csv(f, index=False)

# import again
#with zipfile.ZipFile(zip_filename, 'r') as zf:
#    with zf.open(output_path) as f:
#        df = pd.read_csv(f)


KeyboardInterrupt: 

In [31]:
# Simulate a large DataFrame with 22 million rows (for testing purposes)
num_rows = len(df)
chunk_size = 1_000_000  # Define chunk size (e.g., 1M rows per CSV)


# Define output ZIP file
zip_filename = "output/splitted_politics_data.zip"
#os.makedirs("output", exist_ok=True)  # Ensure output directory exists

# Create ZIP file and write chunks
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, allowZip64=True) as zf:
    for i, chunk in enumerate(range(0, num_rows, chunk_size)):
        chunk_df = df.iloc[chunk:chunk + chunk_size]  # Slice the DataFrame
        csv_name = f"data_part_{i+1}.csv"

        # Convert DataFrame to CSV and store in memory (avoid large temp files)
        csv_data = chunk_df.to_csv(index=False).encode('utf-8')

        # Write CSV data to ZIP
        with zf.open(csv_name, 'w') as f:
            f.write(csv_data)

print(f"Successfully saved {num_rows} rows in {zip_filename} with {num_rows//chunk_size + 1} parts.")

Successfully saved 22178094 rows in output/splitted_politics_data.zip with 23 parts.


In [33]:
df['exact_time_child'][0]

Timestamp('2007-08-06 16:47:47')

In [34]:
df[df['exact_time_child'] >= "2014-01-01"]


,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
313,c2g3ig,erjstov,"As a reminder, this subreddit is for civil di...",politics,2019-06-19 12:00:59,AutoModerator,None,1,None,2g129,...,wockyman,4,4,None,2007-08-17 15:21:20,,neoronin,None,144,2007-08-17 11:44:44
314,c2g3ig,erjt073,Smart move by TeamJoe. I think all the candida...,politics,2019-06-19 12:04:06,This_is_Ed_Chambers,None,0,None,2g129,...,wockyman,4,4,None,2007-08-17 15:21:20,,neoronin,None,144,2007-08-17 11:44:44
521,c2i0aq,erk5hrv,"As a reminder, this subreddit is for civil di...",politics,2019-06-19 14:49:50,AutoModerator,None,1,None,2hzc9,...,aeon2012,-5,-5,None,2007-08-23 16:41:22,,Bestman0,None,121,2007-08-23 15:32:15
522,c2i0aq,erkdukl,Is there some legislative mechanism in which t...,politics,2019-06-19 16:21:09,muskieguy13,None,4,None,2hzc9,...,aeon2012,-5,-5,None,2007-08-23 16:41:22,,Bestman0,None,121,2007-08-23 15:32:15
740,erkh0re,erkhjkp,gullible folks cult 45 members....period,politics,2019-06-19 17:00:04,Getoffmytruthcloud,None,8,None,c2jhw3,...,_rock_farmer,None,22,None,2019-06-19 16:54:37,,1PositiveKarma1,None,3,2019-06-19 16:53:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22178089,kfrhv0b,kfrksgo,The Department of Justice does not consider th...,politics,2023-12-31 23:31:18,Ok-Conversation2707,36,36,0,18vizm7,...,theslats,15,15,0,2023-12-31 23:10:26,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178090,kfrjbu1,kfrn1f1,Texas literally puts all their newly released ...,politics,2023-12-31 23:46:51,Helping-ways,5,5,0,18vizm7,...,StopLookListenNow,3,3,0,2023-12-31 23:20:55,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178091,kfrh2km,kfrncg6,And arresting those who drive the busesplanes....,politics,2023-12-31 23:49:00,milehigh73a,105,105,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11
22178092,kfrh2km,kfrojav,"Why dont they do this? For real, is there a la...",politics,2023-12-31 23:57:21,aitamailmaner,-1,-1,0,18vizm7,...,Fun-Persimmon1207,189,189,0,2023-12-31 23:04:37,,TheTruthTalker800,3035,3035,2023-12-31 22:40:11


***
## **Descriptive Stats**

In [23]:
# get nr of users

comm_auth = list(df_comments['author'].unique())
sub_auth = list(df_submissions['author'].unique())

authors = comm_auth + sub_auth
print(len(authors))
authors = set(authors)
authors.remove('[deleted]')
print("Unique Users: ", len(set(authors)))

4360721
Unique Users:  4044721


### Time Stats and originally pulled counts

In [24]:
df_comments_replies

,author,created_utc,parent_id,subreddit_id,body,link_id,name,id,ups,score,downs,subreddit,likes,replies,stickied,send_replies,comment_type,date
0,lojomofo,1.186380e+09,2cnnc,2cneq,"DAMN, she's a fox! Also I love his retort to t...",2cnnc,None,c2cnq9,6,6,None,politics,None,None,False,None,None,2007-08-06
1,[deleted],1.186380e+09,2cnnc,2cneq,[removed],2cnnc,None,c2cnqr,0,0,None,politics,None,None,False,None,None,2007-08-06
2,shadowsurge,1.186382e+09,2cnti,2cneq,"Personally I'd prefer both, its just that I ha...",2cnti,None,c2cnwe,12,12,None,politics,None,None,False,None,None,2007-08-06
3,wats8976,1.186383e+09,2cnvc,2cneq,What no?,2cnvc,None,c2cnxw,-1,-1,None,politics,None,None,False,None,None,2007-08-06
4,cl3ft,1.186383e+09,2cnvc,2cneq,"well thought out argument there, nigh on irref...",2cnvc,None,c2cnyl,4,4,None,politics,None,None,False,None,None,2007-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187786789,sunflowerastronaut,1.704067e+09,kfrnwmw,2cneq,Experts believe the bill will reduce emissions...,18vd4is,kfrou7r,kfrou7r,2,2,0,politics,None,,False,True,None,2023-12-31
187786790,Tario70,1.704067e+09,kfrhll8,2cneq,Hence why I said it’s a shit situation all aro...,18usgjt,kfrouhj,kfrouhj,1,1,0,politics,None,,False,True,None,2023-12-31
187786791,enigmaroboto,1.704067e+09,kfown66,2cneq,Exactly,18usgjt,kfrouno,kfrouno,-2,-2,0,politics,None,,False,True,None,2023-12-31
187786792,ScaryBuilder9886,1.704067e+09,kfrel1k,2cneq,I don't think that's a change in the definitio...,18vgahm,kfrouuj,kfrouuj,1,1,0,politics,None,,False,True,None,2023-12-31


In [25]:
# Time frame
print("Time Frame \n---------------------------------------------")
print(f"Beginning of time frame: {df_comments_replies['date'].min()}")
print(f"End of time frame: {df_comments_replies['date'].max()}")
print("---------------------------------------------\n")

print("Number of Submissions, Comments and Replies \n---------------------------------------------")
print(f"Number of submissions: {len(submissions)}")
print(f"Number of comments and replies: {len(df_comments_replies)}")
print("---------------------------------------------\n")


# irrelevant, because submissions need to be filtered and matched to comments and replies

# print("Number of pure comments and first level replies \n---------------------------------------------")
# print(f"Number of comments: {len(comments)}")
# print(f"Number of replies: {len(replies)}")
# print(f"Sum of both: {len(comments) + len(replies)}")
# print("---------------------------------------------\n")

# print("Number of lost replies \n---------------------------------------------")
# print(f"Lost replies: {len(df_comments_replies) - (len(comments) + len(replies))}")
# print("---------------------------------------------\n")

Time Frame 
---------------------------------------------
Beginning of time frame: 2007-08-06
End of time frame: 2023-12-31
---------------------------------------------

Number of Submissions, Comments and Replies 
---------------------------------------------
Number of submissions: 4609228
Number of comments and replies: 187786794
---------------------------------------------



In [26]:
df_comments_replies["tokens"] = df_comments_replies["body"].apply(lambda x: x.split())
df_comments_replies["word_count"] = df_comments_replies["tokens"].apply(len)

print(df_comments_replies['word_count'].min())
print(df_comments_replies['word_count'].max())
print(df_comments_replies['word_count'].value_counts())

fig, ax = plt.subplots()
ax.hist(df_comments_replies['word_count'], bins = 20, range = (0, 600))
ax.set_title("Histogram of Word Count")
#ax.set_xticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]) 
#ax.set_xticklabels(["0.0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1"])  
ax.set_xlabel("Word Counts")
ax.set_ylabel("Frequency")
ax.grid(True)

KeyboardInterrupt: 

### Descriptive Stats after Matching

In [27]:
# Descriptive stats after matching

print("Number of unique replies: ", len(df_matched['msg_id_child'].unique()))
print("Number of unique comments: ", len(df_matched['msg_id_parent'].unique()))
print("Number of unique submissions: ", len(df_matched['submission_id'].unique()))

Number of unique replies:  31541402
Number of unique comments:  11780474
Number of unique submissions:  556646


### Descriptive Stats after cleaning for high quality texts

In [28]:
print("Unique Replies: ", len(df['msg_id_child'].unique()))
print("Unique Comments: ", len(df['msg_id_parent'].unique()))

Unique Replies:  22178083
Unique Comments:  8820301
